# Model recovery attack: `credit-card`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

df = pd.read_excel('../datasets/credit-card.xls', header=1, index_col=0).sample(frac=1)
min_values = df.drop(columns=["default payment next month"]).describe().transpose()['min'].to_numpy()
max_values = df.drop(columns=["default payment next month"]).describe().transpose()['max'].to_numpy()
x = df.drop(columns=["default payment next month"]).to_numpy()
x = (x-min_values)/(max_values-min_values)
y = df["default payment next month"].to_numpy().reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-02-16 01:17:42.984123: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-16 01:17:42.984185: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpg2
2022-02-16 01:17:42.984196: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpg2
2022-02-16 01:17:42.984331: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-02-16 01:17:42.984366: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-02-16 01:17:42.984375: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.103.1
2022-02-16 01:17:42.984912: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in per

In [3]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(xin, output)

In [4]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.binary_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=500
    )

Iteration 0001: Training loss: 0.6923
Iteration 0501: Training loss: 0.5262
Iteration 1001: Training loss: 0.4391
Iteration 1501: Training loss: 0.3969
Iteration 2001: Training loss: 0.4505
Iteration 2501: Training loss: 0.4537
Iteration 3001: Training loss: 0.3641
Iteration 3501: Training loss: 0.4473
Iteration 4001: Training loss: 0.4087
Iteration 4501: Training loss: 0.3599


In [5]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(23, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [6]:
log_attack = mra.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.2216, RG-normal: 0.2047, reconstruction validation: 0.0622
Iteration 0002: RG-uniform: 0.2134, RG-normal: 0.2178, reconstruction validation: 0.0383
Iteration 0003: RG-uniform: 0.2210, RG-normal: 0.1950, reconstruction validation: 0.0320
Iteration 0004: RG-uniform: 0.2451, RG-normal: 0.2114, reconstruction validation: 0.0356
Iteration 0005: RG-uniform: 0.2496, RG-normal: 0.2222, reconstruction validation: 0.0203
Iteration 0006: RG-uniform: 0.2175, RG-normal: 0.2010, reconstruction validation: 0.0309
Iteration 0007: RG-uniform: 0.2335, RG-normal: 0.2166, reconstruction validation: 0.0181
Iteration 0008: RG-uniform: 0.2375, RG-normal: 0.2001, reconstruction validation: 0.0164
Iteration 0009: RG-uniform: 0.2258, RG-normal: 0.1972, reconstruction validation: 0.0083
Iteration 0010: RG-uniform: 0.2453, RG-normal: 0.2118, reconstruction validation: 0.0171
Iteration 0011: RG-uniform: 0.2294, RG-normal: 0.2145, reconstruction validation: 0.0119
Iteration 0012: RG-un

In [7]:
np.savetxt("./log/credit-3.csv", log_attack, delimiter=",")